In [4]:
import re
import requests
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup as soup
from typing import List, Union, Dict

In [20]:
periods = [
    '2003-11',
    '2000-03',
    '2002'
]

In [15]:
import numpy as np
from KYOTO.libraries.dst import DstExtraction
classification_rules = {
    'fraca':            np.array(range(-31, -51, -1)),
    'moderada':         np.array(range(-51, -101, -1)),
    'intensa':          np.array(range(-101, -251, -1)),
    'super_intensa':    np.array(range(-251, -1001, -1)),
}

dst = DstExtraction(periods=periods)
dst.make_classification(classification_rules)

In [ ]:
import numpy as np
from KYOTO.libraries.kp import KpExtraction

kp = KpExtraction(periods=periods)

In [16]:
df_filtered = dst.df.copy()

In [7]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.preprocessing import LabelEncoder

In [8]:
def class_count(df: pd.DataFrame):
    count = {"fraca": 0, "moderada": 0, "intensa": 0, "super_intensa": 0}
    for i in range(1, len(df)):
        count[df['classification'].iloc[i]]+=1
    print(count)

In [19]:
class_count(df_filtered)

{'fraca': 119, 'moderada': 69, 'intensa': 18, 'super_intensa': 2}


In [9]:
def accuracy_range(results):
    mean = results.mean()
    dv = results.std()
    print('Acurácia média: {:.2f}%'.format(mean*100))
    print('Intervalo de acurácia: [{:.2f}% ~ {:.2f}%]'.format((mean - 2*dv)*100, (mean + 2*dv)*100))

In [10]:
def intervalo_prec(results):
  for i in range(1, len(results)):
    mean = results.mean()
    dv = results.std()
    print('Precisão média: {:.2f}%'.format(mean*100))
    print('Intervalo de Precisão: [{:.2f}% ~ {:.2f}%]'.format((mean - 2*dv)*100, (mean + 2*dv)*100))

In [11]:
le = LabelEncoder()
df_filtered['classification'] = le.fit_transform(df_filtered['classification'])

# divisao entre treino e teste do dataframe original
X = df_filtered['dst_min'].values.reshape(-1, 1) 
y = df_filtered['classification']

SEED = 10
np.random.seed(SEED)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, stratify = y)
print('O dataset de treino possui {} tempestades e o de teste {} tempestades.'.format(X_train.shape[0], X_test.shape[0]))


cv = StratifiedKFold(n_splits=5, shuffle=True)
model = GaussianNB()
accuracy = cross_val_score(model, X_train, y_train, cv = cv, scoring='accuracy')
#precision = cross_val_score(model, X_train, y_train, cv = cv, scoring='precision')

accuracy_range(accuracy)
#precision_range(precision)

O dataset de treino possui 146 tempestades e o de teste 63 tempestades.
Acurácia média: 96.60%
Intervalo de acurácia: [90.53% ~ 102.67%]


/home/gabriel/tcc/venv/lib/python3.8/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
